# Build a list of Calspec Star for AuxTel

- author : Sylvie Dagoret-Campagne
- affliliation : CNRS
- creation date : 25/09/2019
- Update : October 8th 2019
- Update : April 20th 2020 for package PhotomCorrectAuxTel


From Pysynphot, il get all CALSPEC filenames. It extract HD type stars, get their Ra Dec and filename and make a summary file of all the CALSPEC available

In [ ]:
import os
import re
import numpy as np
from math import ceil,floor

In [ ]:
import pysynphot as S

In [ ]:
import locale
locale.setlocale(locale.LC_ALL,'en_US.UTF-8') # vary depending on your lang/locale

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import time
from datetime import datetime,date
import dateutil.parser
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy.coordinates import Angle
from astropy import units as u
import astropy.coordinates as coord

In [ ]:
from astropy.io import fits
from astropy.table import Table
from astropy.time import Time
import astropy
from astropy.visualization.mpl_normalize import (ImageNormalize,MinMaxInterval,PercentileInterval,SqrtStretch,LogStretch)
from astropy.visualization.wcsaxes import SphericalCircle

In [ ]:
from astroquery.simbad import Simbad

In [ ]:
pysynphot_root_path=os.environ['PYSYN_CDBS']

In [ ]:
pysynphot_root_path

In [ ]:
path_sed_calspec=os.path.join(pysynphot_root_path,'calspec')

In [ ]:
def GetListOfFiles(thedir,tag):
    """
    - thedir : directory where are the files
    - tage  = the re expression to file the filename with regular expression
    """
    all_files=os.listdir(thedir)
    sorted_files=sorted(all_files)
    selected_files=[]
    for sfile in sorted_files:
        if re.search(tag,sfile):
            selected_files.append(sfile)
    return selected_files

In [ ]:
def FilterListOfFiles(listOfFiles):
    """
    Filter list of files   
    """
    
    all_selected_files=[]
    
    current_root_fn=None  # root of filename ex hd000000
    current_fn=None    # filename of calspec ex hd000000_stis.fits
    
    for fn in listOfFiles:
        
        root_fn=fn.split("_")[0]
        
        if current_root_fn==None:
            current_root_fn=root_fn
            current_fn=fn
            continue
            
        if root_fn != current_root_fn:
            all_selected_files.append(current_fn)
            
        current_fn=fn
        current_root_fn=root_fn

    return all_selected_files

In [ ]:
# the tag to find the good filename
RETAGFNNAME=".*fits$"
#RETAGFNNAME="^hd.*_stis_[0-9][0-9][0-9].fits$"
#RETAGDRNAME="(^T1M_.*_HD116405_Filtre_None)_bin1x1.*_red.fit$"

In [ ]:
all_files=os.listdir(path_sed_calspec)

In [ ]:
all_files=sorted(all_files)

In [ ]:
print(all_files)

In [ ]:
all_calspec=GetListOfFiles(path_sed_calspec,RETAGFNNAME)

In [ ]:
print(all_calspec)

In [ ]:
all_selected_calspec=FilterListOfFiles(all_calspec)

In [ ]:
print(all_selected_calspec)

In [ ]:
Simbad.list_votable_fields()

In [ ]:
Simbad.add_votable_fields('sptype')
Simbad.add_votable_fields('flux(U)') # add the Vega unit Magnitude 
Simbad.add_votable_fields('flux(B)') # add the Vega unit Magnitude 
Simbad.add_votable_fields('flux(V)') # add the Vega unit Magnitude 
Simbad.add_votable_fields('flux(R)') # add the Vega unit Magnitude 
Simbad.add_votable_fields('flux(I)') # add the Vega unit Magnitude 
#Simbad.add_votable_fields('flux_unit(V)')
#Simbad.add_votable_fields('flux_unit(I)')
#Simbad.add_votable_fields('flux_system(V)')
#Simbad.add_votable_fields('flux_system(I)')
#Simbad.add_votable_fields('ubv')  # Johnson UBV system

In [ ]:
Simbad.get_field_description("otype")

In [ ]:
Simbad.get_field_description("ubv")

In [ ]:
Simbad.get_field_description("uvby")

In [ ]:
all_tables=[]
all_objnames=[]
all_ra=[]
all_dec=[]
all_sp_type=[]
all_sp_qual=[]
all_mag_U=[]
all_mag_B=[]
all_mag_V=[]
all_mag_R=[]
all_mag_I=[]

all_goodcalspecfile=[]

for file in all_selected_calspec:
    fullfilename=os.path.join(path_sed_calspec,file)
    
    hdu = fits.open(fullfilename)
    img=hdu[0].data
    hd=hdu[0].header
    
    print(file)
    
    if file in ["WDcovar_001.fits"]:
        print(">>>>> SKIP file {} ".format(file))
        continue
    
    OBJNAME=hd["TARGETID"]
    print("==============",OBJNAME,"===========================")
    
    if OBJNAME in ["1732526","1740346","1743045","1757132","1802271","1805292","1808347","1812095","1812524","2M0036+18","2M0559-14","AGK+81D266",
                   "BD02D3375","BD17D4708","BD21D0607","BD26D2606","BD29D2091","BD54D1216","BD60D1753","BD75","C26202","ETAUMA","GJ7541A","GRW+70D5824","HS2027","KF01T5",
                  "KF06T1","KF06T2","KF08T3","KSI2CETI","LAMLEP","MUCOL","P041C","P177D","P330E","SF1615001A","SF1615+001A","SNAP-1","SNAP-2","SUN_REFERENCE","WD0947_857","WD1026_453",
                   "HZ43B","WD0320-539","VB8"]:
        print(">>>>> SKIP TARGET {} ".format(OBJNAME))
        continue
        
    result_table = Simbad.query_object(OBJNAME)
    
    
    if len(result_table)==0:
        print(">>>>> TARGET {} not found in Simbad".format(OBJNAME))
        continue
        
    
           
    result_table.pprint(show_unit=True, max_width=80)
    
    ra = coord.Angle(result_table["RA"], unit=u.hour) # create an Angle object
    dec = coord.Angle(result_table["DEC"], unit=u.deg) # create an Angle object
    skycoord=SkyCoord(ra=ra,dec=dec)[0]
    
    thetype=result_table["SP_TYPE"][0]
    thequal=result_table["SP_QUAL"][0]
    #mag_V_vega=result_table["FLUX_V_2"][0]
    mag_U_vega=result_table["FLUX_U"][0]
    mag_B_vega=result_table["FLUX_B"][0]
    mag_V_vega=result_table["FLUX_V"][0]
    mag_R_vega=result_table["FLUX_R"][0]
    mag_I_vega=result_table["FLUX_I"][0]
        
    all_objnames.append(OBJNAME)
    all_tables.append(result_table)
    all_ra.append(skycoord.ra.degree)
    all_dec.append(skycoord.dec.degree)
    all_sp_type.append(thetype)
    all_sp_qual.append(thequal)
    all_mag_U.append(mag_U_vega)
    all_mag_B.append(mag_B_vega)
    all_mag_V.append(mag_V_vega)
    all_mag_R.append(mag_R_vega)
    all_mag_I.append(mag_I_vega)
    all_goodcalspecfile.append(file)

# Create an astropy table to save good calspec

In [ ]:
t = Table()

In [ ]:
t["OBJNAME"]=all_objnames
t["RA"]=all_ra
t["DEC"]=all_dec
t["SP_TYPE"]=all_sp_type
t["SP_QUAL"]=all_sp_qual
t["magU"]=all_mag_U
t["magB"]=all_mag_B
t["magV"]=all_mag_V
t["magR"]=all_mag_R
t["magI"]=all_mag_I
t["FILES"]=all_goodcalspecfile


In [ ]:
t["RA"].info.format ='5.3f'
t["DEC"].info.format ='5.3f'
t["magU"].info.format ='5.3f'
t["magB"].info.format ='5.3f'
t["magV"].info.format ='5.3f'
t["magR"].info.format ='5.3f'
t["magI"].info.format ='5.3f'

In [ ]:
t.show_in_notebook()  

In [ ]:
t.write('table_summary_allcalspec.fits', overwrite=True)  